## Семинар 13: Efficiency

## План

1. Factorization: Low-Rank Adaptation (LoRA)
2. Distillation: (Hinton KD, Zagoruyko AT)
3. Quantization: AWQ, AQLM, bitsandbytes

## 0. Ключи

In [ ]:
!pip install transformers accelerate evaluate datasets peft bitsandbytes wandb -q

In [ ]:
import os
import wandb

%config InlineBackend.figure_format='retina'

In [ ]:
wandb.login()

## 1. Factorization: Low-Rank Adaptation (LoRA)

В этом блоке посмотрим, как использовать [LoRA](https://arxiv.org/abs/2106.09685) из 🤗 PEFT для файнтюнинга модели SegFormer для семантической сегментации, используя только небольшую долю исходных обучаемых параметров модели.

LoRA добавляет низкоранговые "матрицы обновления" к определенным блокам базовой модели (в данном случае к блокам внимания) и обучает только эти матрицы во время файнтюнинга.

Во время инференса эти матрицы обновления _сливаются_ с исходными параметрами модели. Более подробно можно почитать в [оригинальной статье LoRA](https://arxiv.org/abs/2106.09685).

<div>
<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png" width=70%/>
</div>

### MVP алгоритма

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
from typing import Optional, List


class LoRALayer:
    def __init__(
        self,
        r: int,
        lora_alpha: int,
        lora_dropout: float,
        merge_weights: bool,
    ):
        self.r = r
        self.lora_alpha = lora_alpha
        # Optional dropout
        if lora_dropout > 0.0:
            self.lora_dropout = nn.Dropout(p=lora_dropout)
        else:
            self.lora_dropout = lambda x: x
        # Mark the weight as unmerged
        self.merged = False
        self.merge_weights = merge_weights


class Linear(nn.Linear, LoRALayer):
    # LoRA implemented in a dense layer
    def __init__(
        self,
        in_features: int,
        out_features: int,
        r: int = 0,
        lora_alpha: int = 1,
        lora_dropout: float = 0.0,
        fan_in_fan_out: bool = False,  # Set this to True if the layer to replace stores weight like (fan_in, fan_out)
        merge_weights: bool = True,
        **kwargs,
    ):
        nn.Linear.__init__(self, in_features, out_features, **kwargs)
        LoRALayer.__init__(self, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout, merge_weights=merge_weights)

        self.fan_in_fan_out = fan_in_fan_out
        # Actual trainable parameters
        if r > 0:
            ## YOUR CODE HERE ##
            # lora_A, lora_B, scaling...
            # Freezing the pre-trained weight matrix
            self.weight.requires_grad = False
        self.reset_parameters()
        if fan_in_fan_out:
            self.weight.data = self.weight.data.transpose(0, 1)

    def reset_parameters(self):
        nn.Linear.reset_parameters(self)
        if hasattr(self, "lora_A"):
            # initialize A the same way as the default for nn.Linear and B to zero
            # this is different than what is described in the paper but should not affect performance
            nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
            nn.init.zeros_(self.lora_B)

    def train(self, mode: bool = True):
        def T(w):
            return w.transpose(0, 1) if self.fan_in_fan_out else w

        nn.Linear.train(self, mode)
        if mode:
            if self.merge_weights and self.merged:
                # Make sure that the weights are not merged
                if self.r > 0:
                    self.weight.data -= T(self.lora_B @ self.lora_A) * self.scaling
                self.merged = False
        else:
            if self.merge_weights and not self.merged:
                # Merge the weights and mark it
                if self.r > 0:
                    self.weight.data += T(self.lora_B @ self.lora_A) * self.scaling
                self.merged = True

    def forward(self, x: torch.Tensor):
        def T(w):
            return w.transpose(0, 1) if self.fan_in_fan_out else w

        ## YOUR CODE HERE ##
        # W [out, in], x [B, in] -> W.T @ x
        # Wx + B @ A @ dropout(x)...

        ## YOUR CODE HERE ##
        return result

In [ ]:
B = 15
in_features, out_features = 40, 70

test_tensor = torch.randn(B, in_features)

lora_linear_layer = Linear(in_features, out_features, r=16, fan_in_fan_out=True)

In [ ]:
torch.cuda.device_count()

In [ ]:
lora_linear_layer.lora_A.shape, lora_linear_layer.lora_B.shape, lora_linear_layer.fan_in_fan_out

In [ ]:
lora_linear_layer(test_tensor).shape

In [ ]:
lora_linear_layer.training

### Загрузка датасета

Используем только первые 150 экземпляров из трейнсета [SceneParse150](https://huggingface.co/datasets/scene_parse_150), чтобы уложиться в семинар.

In [ ]:
from datasets import load_dataset

ds = load_dataset("scene_parse_150", split="train[:150]")

### Трейн и тест

In [ ]:
ds = ds.train_test_split(test_size=0.1)
train_ds = ds["train"]
test_ds = ds["test"]

### Лейблы и id

Мы создаем два словаря:

* `label2id`: отображает семантические классы набора данных в целочисленные идентификаторы.
* `id2label`: `label2id` наоборот.

In [ ]:
import json
from huggingface_hub import cached_download, hf_hub_url

repo_id = "huggingface/label-files"
filename = "ade20k-id2label.json"
id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

### Трансформы и аугментации

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, do_reduce_labels=True)

In [ ]:
from torchvision.transforms import ColorJitter

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(ds["train"][1]["image"])
plt.show()
plt.imshow(ds["train"][1]["annotation"], cmap="gray")

In [ ]:
from PIL import Image
import numpy as np


def handle_grayscale_image(image):
    np_image = np.array(image)
    if np_image.ndim == 2:
        tiled_image = np.tile(np.expand_dims(np_image, -1), 3)
        return Image.fromarray(tiled_image)
    else:
        return Image.fromarray(np_image)


def train_transforms(example_batch):
    images = [jitter(handle_grayscale_image(x)) for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [handle_grayscale_image(x) for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

In [ ]:
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

### Оценка качества

Включим мониторинг качества во время обучения. Загрузить метод оценки с помощью библиотеки [🤗 Evaluate](https://huggingface.co/docs/evaluate/index). Для сегментации подойдёт [mean Intersection over Union (IoU)](https://huggingface.co/spaces/evaluate-metric/accuracy) (см. 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour), чтобы узнать больше о том, как загрузить и вычислить метрику):

In [ ]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")


def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        # scale the logits to the size of the label
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        # currently using _compute instead of compute
        # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=image_processor.do_reduce_labels,
        )

        # add per category metrics as individual key-value pairs
        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

        return metrics

### Загрузим модель

Будем использовать и так не очень большой [SegFormer B0 variant](https://huggingface.co/nvidia/mit-b0).

In [ ]:
def get_trainable_parameters(model):
    """
    Counts the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"

Мы передаем словари `label2id` и `id2label`, чтобы класс `AutoModelForSemanticSegmentation` знал, что нас интересует пользовательская базовая модель, в которой головка декодера должна быть случайным образом инициализирована с учетом нашего пользовательского набора данных. Заметим, однако, что остальные параметры модели предварительно обучены и будут настраиваться в рамках обычного трансферного обучения.

Видим, что 100% параметров в `model` обучаемые.

In [ ]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer

model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)
get_trainable_parameters(model)

### Оборачиваем `model` в `PeftModel` для обучения с LoRA

Это включает в себя два шага:

* Определение конфигурации с помощью `LoraConfig`
* Обертывание исходной `модели с помощью `get_peft_model()` с конфигурацией, определенной в шаге выше.

In [ ]:
LoraConfig?

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="lora_only",
    modules_to_save=["decode_head"],
)

lora_model = get_peft_model(model, config)
get_trainable_parameters(lora_model)

Давайте разберемся, что здесь происходит.

Для того чтобы LoRA начала действовать, нам нужно указать целевые модули в `LoraConfig`, чтобы `PeftModel` знал, какие модули внутри нашей модели нужно дополнить матрицами LoRA. В данном случае нас интересуют только матрицы запросов и значений блоков внимания базовой модели. Поскольку параметры, соответствующие этим матрицам, «названы» `query` и `value` соответственно, мы указываем их соответствующим образом в аргументе `target_modules` в `LoraConfig`.

Мы также указываем `modules_to_save`. После того, как мы обернули нашу базовую модель `model` в `PeftModel` вместе с `config`, мы получаем новую модель, в которой обучаемыми являются только параметры LoRA (так называемые «матрицы обновления»), а предварительно обученные параметры остаются замороженными. К ним относятся и параметры случайно инициализированного классификатора. Этого мы не хотим при тонкой настройке базовой модели на нашем пользовательском наборе данных. Чтобы гарантировать, что параметры классификатора также будут обучены, мы указываем `modules_to_save`. Это также гарантирует, что эти модули будут сериализованы вместе с обучаемыми параметрами LoRA при использовании таких утилит, как `save_pretrained()` и `push_to_hub()`.  

Что касается других параметров:

* `r`: Размерность, используемая матрицами обновления LoRA.
* `alpha`: Масштабирующий фактор.
* `bias`: Указывает, нужно ли обучать параметры `bias`. `lora_only` означает, что будут обучаться только параметры `bias` LoRA.

`r` и `alpha` вместе контролируют общее количество конечных обучаемых параметров при использовании LoRA, предоставляя нам гибкость в поиске компромисса между конечной производительностью и эффективностью вычислений.

Мы также можем определить, сколько параметров мы на самом деле обучаем. Поскольку мы заинтересованы в **parameter-efficient fine-tuning**, мы должны ожидать, что количество обучаемых параметров в `lora_model` будет меньше по сравнению с исходной `model`, что и происходит.

Чтобы не ошибиться, давайте также вручную проверим модули, которые действительно поддаются обучению в `lora_model`.

In [ ]:
all_params = []

for name, param in model.named_parameters():
    all_params.append((name, param.shape))
print(*grad_params, sep="\n")

In [ ]:
grad_params = []

for name, param in lora_model.named_parameters():
    if param.requires_grad:
        grad_params.append((name, param.shape))

In [ ]:
print(*grad_params, sep="\n")

Мы можем подтвердить, что только параметры с LoRA, добавляемые к блокам внимания, и параметры `decode_head` теперь обучаемы.

### Тренируем!

Это двухэтапный процесс:

1. Определите гиперпараметры обучения в [TrainingArguments](https://huggingface.co/docs/transformers/v4.26.0/en/main_classes/trainer#transformers.TrainingArguments). Важно не удалять неиспользуемые столбцы, потому что это приведет к исчезновению столбца изображения. Без столбца изображения вы не сможете создать `pixel_values`. Установите `remove_unused_columns=False`, чтобы предотвратить такое поведение! Единственный другой необходимый параметр - output_dir, который указывает, куда сохранять модель. В конце каждой эпохи `Trainer` будет оценивать метрику IoU и сохранять контрольную точку обучения.
2. Передайте аргументы обучения в [Trainer](https://huggingface.co/docs/transformers/v4.26.0/en/main_classes/trainer#transformers.Trainer) вместе с моделью, набором данных, токенизатором, коллатором данных и функцией `compute_metrics`.
3. Вызовите `train()` для тренировки модели.


**Примечание**: этот пример предназначен для ознакомления с рабочим процессом при использовании PEFT для семантической сегментации. Мы не проводили обширную настройку гиперпараметров для достижения оптимальных результатов.

In [ ]:
import gc


model_name = checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-scene-parse-{len(ds['train'])}-lora",
    learning_rate=5e-4,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    dataloader_num_workers=8,
    per_device_eval_batch_size=4,
    save_total_limit=3,
    evaluation_strategy="steps",
    eval_steps=100,
    warmup_ratio=0.05,
    save_strategy="epoch",
    logging_steps=5,
    bf16=True,
    remove_unused_columns=False,
    push_to_hub=True,
    label_names=["labels"],
    report_to="wandb",
    seed=1337,
    run_name="b0-lora-baseline",
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

In [ ]:
wandb.init()

In [ ]:
trainer.evaluate()

### Сохранение модели и инференс

Здесь мы используем метод `save_pretrained()` из `lora_model` для локального сохранения *LoRA-only параметров*. Однако вы также можете использовать метод `push_to_hub()` для загрузки этих параметров непосредственно в Hugging Face Hub (как показано [здесь](https://colab.research.google.com/github/huggingface/peft/blob/main/examples/image_classification/image_classification_peft_lora.ipynb)).

In [ ]:
model_id = "segformer-scene-parse-150-lora"
lora_model.save_pretrained(model_id)

Мы видим, что размер параметров только для LoRA составляет всего **2,2 МБ**! Это значительно улучшает переносимость при использовании очень больших моделей.

In [ ]:
!ls -lh {model_id}

Теперь давайте подготовим нашу `inference_model` и запустим её.

In [ ]:
from peft import PeftConfig, PeftModel

config = PeftConfig.from_pretrained(model_id)
model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

# Load the Lora model
inference_model = PeftModel.from_pretrained(model, model_id)

Достаём картинку:

In [ ]:
import requests

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/semantic-seg-image.png"
image = Image.open(requests.get(url, stream=True).raw)
image

Препроцессим"

In [ ]:
# prepare image for the model
encoding = image_processor(image.convert("RGB"), return_tensors="pt")
print(encoding.pixel_values.shape)

Прогоняем предсказание:

In [ ]:
with torch.no_grad():
    outputs = inference_model(pixel_values=encoding.pixel_values)
    logits = outputs.logits

upsampled_logits = nn.functional.interpolate(
    logits,
    size=image.size[::-1],
    mode="bilinear",
    align_corners=False,
)

pred_seg = upsampled_logits.argmax(dim=1)[0]

Визуализируем результаты.

Нам нужна цветовая палитра для визуализации результатов. Здесь мы используем [одну из предоставленных репозиторием TensorFlow Model Garden](https://github.com/tensorflow/models/blob/3f1ca33afe3c1631b733ea7e40c294273b9e406d/research/deeplab/utils/get_dataset_colormap.py#L51).

In [ ]:
def ade_palette():
    """Creates a label colormap used in ADE20K segmentation benchmark.
    Returns:
    A colormap for visualizing segmentation results.
    """
    return np.asarray(
        [
            [0, 0, 0],
            [120, 120, 120],
            [180, 120, 120],
            [6, 230, 230],
            [80, 50, 50],
            [4, 200, 3],
            [120, 120, 80],
            [140, 140, 140],
            [204, 5, 255],
            [230, 230, 230],
            [4, 250, 7],
            [224, 5, 255],
            [235, 255, 7],
            [150, 5, 61],
            [120, 120, 70],
            [8, 255, 51],
            [255, 6, 82],
            [143, 255, 140],
            [204, 255, 4],
            [255, 51, 7],
            [204, 70, 3],
            [0, 102, 200],
            [61, 230, 250],
            [255, 6, 51],
            [11, 102, 255],
            [255, 7, 71],
            [255, 9, 224],
            [9, 7, 230],
            [220, 220, 220],
            [255, 9, 92],
            [112, 9, 255],
            [8, 255, 214],
            [7, 255, 224],
            [255, 184, 6],
            [10, 255, 71],
            [255, 41, 10],
            [7, 255, 255],
            [224, 255, 8],
            [102, 8, 255],
            [255, 61, 6],
            [255, 194, 7],
            [255, 122, 8],
            [0, 255, 20],
            [255, 8, 41],
            [255, 5, 153],
            [6, 51, 255],
            [235, 12, 255],
            [160, 150, 20],
            [0, 163, 255],
            [140, 140, 140],
            [250, 10, 15],
            [20, 255, 0],
            [31, 255, 0],
            [255, 31, 0],
            [255, 224, 0],
            [153, 255, 0],
            [0, 0, 255],
            [255, 71, 0],
            [0, 235, 255],
            [0, 173, 255],
            [31, 0, 255],
            [11, 200, 200],
            [255, 82, 0],
            [0, 255, 245],
            [0, 61, 255],
            [0, 255, 112],
            [0, 255, 133],
            [255, 0, 0],
            [255, 163, 0],
            [255, 102, 0],
            [194, 255, 0],
            [0, 143, 255],
            [51, 255, 0],
            [0, 82, 255],
            [0, 255, 41],
            [0, 255, 173],
            [10, 0, 255],
            [173, 255, 0],
            [0, 255, 153],
            [255, 92, 0],
            [255, 0, 255],
            [255, 0, 245],
            [255, 0, 102],
            [255, 173, 0],
            [255, 0, 20],
            [255, 184, 184],
            [0, 31, 255],
            [0, 255, 61],
            [0, 71, 255],
            [255, 0, 204],
            [0, 255, 194],
            [0, 255, 82],
            [0, 10, 255],
            [0, 112, 255],
            [51, 0, 255],
            [0, 194, 255],
            [0, 122, 255],
            [0, 255, 163],
            [255, 153, 0],
            [0, 255, 10],
            [255, 112, 0],
            [143, 255, 0],
            [82, 0, 255],
            [163, 255, 0],
            [255, 235, 0],
            [8, 184, 170],
            [133, 0, 255],
            [0, 255, 92],
            [184, 0, 255],
            [255, 0, 31],
            [0, 184, 255],
            [0, 214, 255],
            [255, 0, 112],
            [92, 255, 0],
            [0, 224, 255],
            [112, 224, 255],
            [70, 184, 160],
            [163, 0, 255],
            [153, 0, 255],
            [71, 255, 0],
            [255, 0, 163],
            [255, 204, 0],
            [255, 0, 143],
            [0, 255, 235],
            [133, 255, 0],
            [255, 0, 235],
            [245, 0, 255],
            [255, 0, 122],
            [255, 245, 0],
            [10, 190, 212],
            [214, 255, 0],
            [0, 204, 255],
            [20, 0, 255],
            [255, 255, 0],
            [0, 153, 255],
            [0, 41, 255],
            [0, 255, 204],
            [41, 0, 255],
            [41, 255, 0],
            [173, 0, 255],
            [0, 245, 255],
            [71, 0, 255],
            [122, 0, 255],
            [0, 255, 184],
            [0, 92, 255],
            [184, 255, 0],
            [0, 133, 255],
            [255, 214, 0],
            [25, 194, 194],
            [102, 255, 0],
            [92, 0, 255],
        ]
    )

In [ ]:
import matplotlib.pyplot as plt

color_seg = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.uint8)
palette = np.array(ade_palette())

for label, color in enumerate(palette):
    color_seg[pred_seg == label, :] = color
color_seg = color_seg[..., ::-1]  # convert to BGR
color_idx = np.unique(pred_seg)

labels = [id2label[i] for i in color_idx]

img = np.array(image) * 0.5 + color_seg * 0.5  # plot the image with the segmentation map
img = img.astype(np.uint8)


In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.barh(
    y=np.arange(len(color_idx)) * 10 + 10,
    height=np.ones(len(color_idx)) * 10,
    width=10,
    tick_label=labels,
    align="center",
    color=palette[color_idx] / 255.0,
)
plt.show()

Результаты определенно не соответствуют ожиданиям, и, как уже говорилось выше, этот пример не предназначен для создания самой современной модели. Он создан для того, чтобы ознакомить вас со всем процессом работы.

С другой стороны, если выполнить полную тонкую настройку на той же установке (тот же вариант модели, тот же набор данных, тот же график обучения и т. д.), результаты не будут отличаться. Это важный аспект параметрически эффективной тонкой настройки - быть в состоянии соответствовать результатам полной тонкой настройки, но с частью всех обучаемых параметров.

Вот некоторые вещи, которые вы можете попробовать, чтобы получить лучшие результаты:

* Увеличить количество данных.
* Попробовать более крупный вариант предобученной модели SegFormer (о доступных вариантах модели можно узнать [здесь](https://huggingface.co/models?search=segformer)).
* Тюнить значения аргументов, доступных в `LoraConfig`.
* Тюнить обычные параметры обучения.
* LoRA можно инициализировать из разложений исходной матрицы, например, SVD в [PiSSA](https://arxiv.org/pdf/2404.02948)

<div>
<img src="https://pbs.twimg.com/media/GKrHjNlaUAIeKNp?format=jpg&name=medium" width=70%/>
</div>


## 2. Distillation: (Hinton KD, Zagoruyko AT)

In [ ]:
! pip install torchdistill -q

In [ ]:
import numpy as np
import random
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchdistill.core.forward_hook import ForwardHookManager

import matplotlib.pyplot as plt

%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args = {}
kwargs = {}
args["batch_size"] = 1000
args["test_batch_size"] = 1000
args["epochs"] = 5  # The number of Epochs is the number of times you go through the full dataset.
args["lr"] = 0.1  # Learning rate is how fast it will decend.
args[
    "momentum"
] = 0.5  # SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args["seed"] = 1  # random seed
args["log_interval"] = 60
args["cuda"] = True

In [ ]:
# fix random seeds
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

g = torch.Generator()
g.manual_seed(0)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
# load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
    ),
    batch_size=args["batch_size"],
    shuffle=True,
    worker_init_fn=seed_worker,
    generator=g,
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data",
        train=False,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]),
    ),
    batch_size=args["batch_size"],
    shuffle=True,
)

In [ ]:
def train(model, epoch, optimizer, args=args):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # This will zero out the gradients for this batch.
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.cross_entropy(output, target)
        # dloss/dx for every Variable
        loss.backward()
        # to do a one-step update on our parameter.
        optimizer.step()
        # Print out the loss periodically.
        if batch_idx % args["log_interval"] == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, args=args):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item()  # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

    return 100.0 * correct / len(test_loader.dataset)

### Тренируем Teacher-а на MNIST

In [ ]:
class TeacherNetClass(nn.Module):
    # This defines the structure of the NN.
    def __init__(self):
        super(TeacherNetClass, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()  # Dropout
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(800, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # Fully Connected Layer/Activation
        x = F.relu(self.fc1(self.flatten(x)))
        x = F.dropout(x, training=self.training)
        # Fully Connected Layer/Activation
        x = self.fc2(x)
        # Softmax gets probabilities.
        return x

In [ ]:
teacher = TeacherNetClass()

In [ ]:
teacher = TeacherNetClass()
teacher.to(device)

optimizer = optim.SGD(teacher.parameters(), lr=args["lr"], momentum=args["momentum"])

for epoch in range(args["epochs"]):
    train(teacher, epoch, optimizer)
    test(teacher)

### Тренируем Studenta-а на MNIST без дистилляции

Мы определяем архитектуру ученика так же, как и архитектуру учителя, но с меньшей шириной в некоторых слоях.



In [ ]:
class StudentNetClass(nn.Module):
    # This defines the structure of the NN.
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()  # Dropout
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(400, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # Fully Connected Layer/Activation
        x = F.relu(self.fc1(self.flatten(x)))
        x = F.dropout(x, training=self.training)
        # Fully Connected Layer/Activation
        x = self.fc2(x)
        # Softmax gets probabilities.
        return x

In [ ]:
student = StudentNetClass()
student.to(device)
acc_list_wo_kd = []

optimizer = optim.SGD(student.parameters(), lr=args["lr"], momentum=args["momentum"])

for epoch in range(args["epochs"]):
    train(student, epoch, optimizer)
    acc_list_wo_kd.append(test(student))

### Тренируем Studenta-а на MNIST с дистилляцией по Хинтону

<div>
<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2023/11/knowledge-distillation-general-framework.png" width=90%/>
</div>

$$L=(1-\alpha) L_{c e}\left(z_{s}, y\right)+\alpha L_{c e}\left(z_{s}, z_{t}\right)$$

In [ ]:
class KDLoss(nn.Module):
    """
    Distilling the Knowledge in a Neural Network
    https://arxiv.org/abs/1503.02531
    """

    def __init__(self, T=1, alpha=0.1):
        super().__init__()
        self.T = T
        self.alpha = alpha

    def forward(self, student_pred, teacher_pred, y):
        """
        Loss function for student network: Loss = alpha * (distillation loss with soft-target) + (1 - alpha) * (cross-entropy loss with true label)
        Return: loss
        """
        hard_loss =  F.cross_entropy(student_pred, y)
        if self.alpha > 0:
            ### YOUR CODE HERE ###
            # soft_loss = ...
            loss = self.alpha * soft_loss + (1 - self.alpha) * hard_loss
        else:
            loss = hard_loss
        return loss

In [ ]:
def train_kd(
    teacher,
    student,
    epoch,
    optimizer,
    kd_loss,
    alpha=0.1,
    args=args,
):
    teacher.to(device)
    teacher.eval()
    student.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # This will zero out the gradients for this batch.
        optimizer.zero_grad()
        output = student(data)

        with torch.no_grad():
            output_t = teacher(data)

        # Calculate KD Loss
        loss = kd_loss(output, output_t, target)
        # dloss/dx for every Variable
        loss.backward()
        # to do a one-step update on our parameter.
        optimizer.step()
        # Print out the loss periodically.
        if batch_idx % args["log_interval"] == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

In [ ]:
alphas = [0, 0.02, 0.07, 0.2, 0.4, 1.0]

exp_dict = {}

for alpha in alphas:
    print(f"__________(alpha = {alpha})__________")
    acc_list = []
    student_hint = StudentNetClass()
    student_hint.to(device)

    # Create forward hooks for outputs of conv1 and conv2 for teacher and student
    kd_loss = KDLoss(T=5, alpha=alpha)

    optimizer = optim.SGD(student_hint.parameters(), lr=args["lr"], momentum=args["momentum"])

    for epoch in range(args["epochs"]):
        train_kd(teacher, student_hint, epoch, optimizer, kd_loss)
        acc_list.append(test(student_hint))

    exp_dict[alpha] = acc_list

In [ ]:
plt.figure(figsize=(12, 8))

for alpha, acc_list in exp_dict.items():
    plt.plot(acc_list, label=f"alpha = {alpha}", marker="o")

plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

### Тренируем Studenta-а на MNIST с дистилляцией по Загоруйко (Attention Transfer)


<div>
<img src="https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/20e210bb6b1d3e637e2b2674aeead3fad8c2c70e/2-Figure1-1.png" width=70%/>
</div>

$$L=(1-\alpha) L_{c e}\left(z_{s}, y\right)+\alpha L_{A T}\left(h_{s}, h_{t}\right)$$

$$L_{A T}=L\left(\mathbf{W}_{S}, x\right)+\frac{\beta}{2} \sum_{j \in I}\left\|\frac{Q_{S}^{j}}{\left\|Q_{S}^{j}\right\|_{2}}-\frac{Q_{T}^{j}}{\left\|Q_{T}^{j}\right\|_{2}}\right\|_{p}$$

In [ ]:
class AT(nn.Module):
    """
    Paying More Attention to Attention: Improving the Performance of Convolutional
    Neural Networks wia Attention Transfer
    https://arxiv.org/pdf/1612.03928.pdf
    """

    def __init__(self, p=2):
        super(AT, self).__init__()
        self.p = p

    def forward(self, fm_s, fm_t):
        loss = torch.norm(self.attention_map(fm_s) - self.attention_map(fm_t), self.p)

        return loss

    def attention_map(self, fm):
        ### YOUR CODE HERE ###
        # am = ...
        return am

In [ ]:
def train_at(
    teacher,
    student,
    epoch,
    optimizer,
    at_loss,
    forward_hook_manager_teacher,
    forward_hook_manager_student,
    alpha=0.1,
    args=args,
):
    device = "cuda" if args["cuda"] else "cpu"
    teacher.to(device)
    teacher.eval()
    student.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        if args["cuda"]:
            data, target = data.to(device), target.to(device)
        # This will zero out the gradients for this batch.
        optimizer.zero_grad()
        output = student(data)

        with torch.no_grad():
            output_t = teacher(data)

        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss_label = F.cross_entropy(output, target)

        # Calculate Attention loss
        teacher_io_dict = forward_hook_manager_teacher.pop_io_dict()
        student_io_dict = forward_hook_manager_student.pop_io_dict()
        loss_at = at_loss(student_io_dict["conv1"]["output"], teacher_io_dict["conv1"]["output"]) + at_loss(
            student_io_dict["conv2"]["output"], teacher_io_dict["conv2"]["output"]
        )

        loss = (1 - alpha) * loss_label + alpha * loss_at
        # dloss/dx for every Variable
        loss.backward()
        # to do a one-step update on our parameter.
        optimizer.step()
        # Print out the loss periodically.
        if batch_idx % args["log_interval"] == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

In [ ]:
alphas = [0, 0.05, 0.07, 0.1, 0.2, 0.4, 0.5, 1.0]

exp_dict = {}

for alpha in alphas:
    print(f"__________(alpha = {alpha})__________")
    acc_list = []
    forward_hook_manager_teacher = ForwardHookManager(device)
    forward_hook_manager_student = ForwardHookManager(device)

    student_at = StudentNetClass()
    student_at.to(device)

    # Create forward hooks for outputs of conv1 and conv2 for teacher and student

    forward_hook_manager_teacher.add_hook(teacher, "conv1", requires_input=False, requires_output=True)
    forward_hook_manager_student.add_hook(student_at, "conv1", requires_input=False, requires_output=True)

    forward_hook_manager_teacher.add_hook(teacher, "conv2", requires_input=False, requires_output=True)
    forward_hook_manager_student.add_hook(student_at, "conv2", requires_input=False, requires_output=True)

    p = 2
    at_loss = AT(p)

    optimizer = optim.SGD(student_at.parameters(), lr=args["lr"], momentum=args["momentum"])

    for epoch in range(args["epochs"]):
        train_at(
            teacher,
            student_at,
            epoch,
            optimizer,
            at_loss,
            forward_hook_manager_teacher,
            forward_hook_manager_student,
            alpha,
        )
        acc_list.append(test(student_at))

    exp_dict[alpha] = acc_list

In [ ]:
# do not forget to close the door
forward_hook_manager_teacher.clear()
forward_hook_manager_student.clear()

In [ ]:
plt.figure(figsize=(12, 8))

for alpha, acc_list in exp_dict.items():
    plt.plot(acc_list, label=f"alpha = {alpha}", marker="o")

plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

for alpha, acc_list in list(exp_dict.items()):
    if alpha != 1.0:
        plt.plot(acc_list, label=f"alpha = {alpha}", marker="o")

plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

## 3. Quantization: AWQ, AQLM, bitsandbytes

- [AWQ](https://huggingface.co/docs/transformers/main/en/quantization/awq): пример квантизации без дотренировки (быстро, но с худшим качеством)
- [AQLM](https://huggingface.co/docs/transformers/main/en/quantization/aqlm), [репо](https://github.com/Vahe1994/AQLM) авторов: пример квантизации с дотренировкой малобитной модели на вашем датасете (требует несколько дней обучения на одной карте, просадка качества минимальна)

Оба метода есть в библиотеке [bitsandbytes](https://huggingface.co/docs/bitsandbytes/main/en/index), подробный [гайд использования](https://huggingface.co/docs/transformers/en/quantization)

In [ ]:
from transformers import BitsAndBytesConfig, AwqConfig, AutoModelForCausalLM

model_id = "microsoft/Phi-3-mini-4k-instruct"
DEVICE = "cuda:0"

quantization_config_bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# quantization_config_awq = AwqConfig(
#     bits=4,
#     fuse_max_seq_len=4096,
#     modules_to_fuse={
#         "attention": ["q_proj", "k_proj", "v_proj", "o_proj"],
#         "mlp": ["gate_proj", "up_proj", "down_proj"],
#         "layernorm": ["input_layernorm", "post_attention_layernorm", "norm"],
#         "use_alibi": False,
#         "num_attention_heads": 32,
#         "num_key_value_heads": 8,
#         "hidden_size": 4096,
#     }
# )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    quantization_config=quantization_config_bnb,
).to(DEVICE)
